# RAG Baseline

In [1]:
import os
import requests
from dotenv import load_dotenv

from langchain_upstage import UpstageDocumentParseLoader
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
# from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_teddynote.community.pinecone import create_index

from rag_utils import logging

# API 키 정보 로드
load_dotenv()

/Users/sean/anaconda3/envs/aitech311/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
# LangSmith 시작
logging.langsmith("test-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
test-RAG


In [3]:
# 단계 1: 문서 로드(Load Documents)  # PyMuPDFLoader
file_path = "./data/politics_and_the_law_page_10-38.pdf"
loader = PyMuPDFLoader(file_path)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 29


In [4]:
# matadata 확인
docs[0].__dict__

{'id': None,
 'metadata': {'source': './data/politics_and_the_law_page_10-38.pdf',
  'file_path': './data/politics_and_the_law_page_10-38.pdf',
  'page': 0,
  'total_pages': 29,
  'format': 'PDF 1.6',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'creator': '',
  'producer': 'PyPDF2',
  'creationDate': '',
  'modDate': '',
  'trapped': ''},
 'page_content': '1\n    \n정치와 법 \n민주 국가에서 선거를 통해 선출된 국민의 대표는 정치 활동을 통해 법을 만들고, 정부는 이러한 \n법에 근거해서 국가를 운영한다. 따라서 국민이 어떤 대표를 뽑느냐에 따라 만들어지는 법도 달라\n지고 국가의 운영 역시 달라진다. 민주 국가에서 국민이 주인이라는 말은 이러한 의미에서 이해할 \n수 있다.\n\u2003\n생각 열기 \n정치권력은 처벌을 가할 수 있는 법률을 만드는 권리를 \n의미한다. 또한 재산을 규제하고 보전하기 위해 강제력을 행\n사할 수 있는 권리이며, 외적의 침입으로부터 국가를 방어하\n기 위해서 무력을 사용하는 권리이다. 그리고 정치권력은 오\n직 공공선을 위해서만 행사하는 권리이다.\n- 로크, 『통치론』\n로크\u2002▶ \n(Locke, John, 1632~1704)\n◀\u2002생피에르\n(Saint-Pierre, Abbé de, 1658~1743)\n법률로 특정한 개인들이 이기적인 이익을 추구\n하는 것을 방지해야 한다. 그렇지 않은 한 개인의 \n이익은 공동의 이익과 갈등을 일으키고 공동체를 \n황폐하게 할 것이다.\n- 생피에르, 『다원 합의제론』\n1  로크에 의하면, 정치의 목적이 무엇일지 생각해 보자.\n2  생피

In [5]:
# 단계 2-1: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 101


In [6]:
dict(split_documents[0])

{'id': None,
 'metadata': {'source': './data/politics_and_the_law_page_10-38.pdf',
  'file_path': './data/politics_and_the_law_page_10-38.pdf',
  'page': 0,
  'total_pages': 29,
  'format': 'PDF 1.6',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'creator': '',
  'producer': 'PyPDF2',
  'creationDate': '',
  'modDate': '',
  'trapped': ''},
 'page_content': '1\n    \n정치와 법 \n민주 국가에서 선거를 통해 선출된 국민의 대표는 정치 활동을 통해 법을 만들고, 정부는 이러한 \n법에 근거해서 국가를 운영한다. 따라서 국민이 어떤 대표를 뽑느냐에 따라 만들어지는 법도 달라\n지고 국가의 운영 역시 달라진다. 민주 국가에서 국민이 주인이라는 말은 이러한 의미에서 이해할 \n수 있다.\n\u2003\n생각 열기 \n정치권력은 처벌을 가할 수 있는 법률을 만드는 권리를 \n의미한다. 또한 재산을 규제하고 보전하기 위해 강제력을 행\n사할 수 있는 권리이며, 외적의 침입으로부터 국가를 방어하\n기 위해서 무력을 사용하는 권리이다. 그리고 정치권력은 오\n직 공공선을 위해서만 행사하는 권리이다.\n- 로크, 『통치론』\n로크\u2002▶ \n(Locke, John, 1632~1704)\n◀\u2002생피에르\n(Saint-Pierre, Abbé de, 1658~1743)\n법률로 특정한 개인들이 이기적인 이익을 추구\n하는 것을 방지해야 한다. 그렇지 않은 한 개인의',
 'type': 'Document'}

In [7]:
for doc in split_documents:
    print(doc.page_content)
    print("+"*100)

1
    
정치와 법 
민주 국가에서 선거를 통해 선출된 국민의 대표는 정치 활동을 통해 법을 만들고, 정부는 이러한 
법에 근거해서 국가를 운영한다. 따라서 국민이 어떤 대표를 뽑느냐에 따라 만들어지는 법도 달라
지고 국가의 운영 역시 달라진다. 민주 국가에서 국민이 주인이라는 말은 이러한 의미에서 이해할 
수 있다.
 
생각 열기 
정치권력은 처벌을 가할 수 있는 법률을 만드는 권리를 
의미한다. 또한 재산을 규제하고 보전하기 위해 강제력을 행
사할 수 있는 권리이며, 외적의 침입으로부터 국가를 방어하
기 위해서 무력을 사용하는 권리이다. 그리고 정치권력은 오
직 공공선을 위해서만 행사하는 권리이다.
- 로크, 『통치론』
로크 ▶ 
(Locke, John, 1632~1704)
◀ 생피에르
(Saint-Pierre, Abbé de, 1658~1743)
법률로 특정한 개인들이 이기적인 이익을 추구
하는 것을 방지해야 한다. 그렇지 않은 한 개인의
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
◀ 생피에르
(Saint-Pierre, Abbé de, 1658~1743)
법률로 특정한 개인들이 이기적인 이익을 추구
하는 것을 방지해야 한다. 그렇지 않은 한 개인의 
이익은 공동의 이익과 갈등을 일으키고 공동체를 
황폐하게 할 것이다.
- 생피에르, 『다원 합의제론』
1  로크에 의하면, 정치의 목적이 무엇일지 생각해 보자.
2  생피에르가 생각하는 법률의 목적을 일상생활의 사례와 관련지어 토의해 보자.
3  ‌‘권력’, ‘법률’, ‘국가’, ‘공익’, ‘개인’ 등의 용어를 사용하여 정치에 관한 짧은 문장을 자유롭게 만들
어 보자.
10 Ⅰ. 민주주의와 헌법
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [8]:
# 단계 2-2: 문서 전처리
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_documents,
    metadata_keys=["source", "page", "author"],
    min_length=5,
    use_basename=True,
)

100%|██████████| 101/101 [00:00<00:00, 135777.15it/s]


In [9]:
contents[0]

'1\n    \n정치와 법 \n민주 국가에서 선거를 통해 선출된 국민의 대표는 정치 활동을 통해 법을 만들고, 정부는 이러한 \n법에 근거해서 국가를 운영한다. 따라서 국민이 어떤 대표를 뽑느냐에 따라 만들어지는 법도 달라\n지고 국가의 운영 역시 달라진다. 민주 국가에서 국민이 주인이라는 말은 이러한 의미에서 이해할 \n수 있다.\n\u2003\n생각 열기 \n정치권력은 처벌을 가할 수 있는 법률을 만드는 권리를 \n의미한다. 또한 재산을 규제하고 보전하기 위해 강제력을 행\n사할 수 있는 권리이며, 외적의 침입으로부터 국가를 방어하\n기 위해서 무력을 사용하는 권리이다. 그리고 정치권력은 오\n직 공공선을 위해서만 행사하는 권리이다.\n- 로크, 『통치론』\n로크\u2002▶ \n(Locke, John, 1632~1704)\n◀\u2002생피에르\n(Saint-Pierre, Abbé de, 1658~1743)\n법률로 특정한 개인들이 이기적인 이익을 추구\n하는 것을 방지해야 한다. 그렇지 않은 한 개인의'

In [10]:
metadatas.keys()

dict_keys(['source', 'page', 'author'])

In [11]:
metadatas['source'][0]

'politics_and_the_law_page_10-38.pdf'

In [12]:
# 문서 개수 확인, 소스 개수 확인, 페이지 개수 확인
len(contents), len(metadatas["source"]), len(metadatas["page"])

(101, 101, 101)

In [13]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = UpstageEmbeddings(model="embedding-passage")  # embedding-query 또는 embedding-passage

In [14]:
# 단계 4: DB 생성(Create DB) 및 저장
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

index_name = "quickstart"
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=4096, 
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")


vectorstore = PineconeVectorStore.from_documents(split_documents, embeddings, index_name=index_name)

quickstart has been successfully created


In [15]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [22]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야.
입력과 관련있는 정보를 참고해서 피드백을 생성해줘.
참고한 정보의 페이지도 같이 알려줘.
만약 틀린 부분이 없을 경우, 칭찬 한문장 작성해줘.

#정보: 
{context}

#입력:
{question}

#답:"""
)

In [23]:
# 단계 7: 언어모델(LLM) 생성
llm = ChatUpstage(model="solar-pro")

In [24]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
chain

{
  context: VectorStoreRetriever(tags=['PineconeVectorStore', 'UpstageEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x11ab26810>, search_kwargs={}),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='너는 입력을 보고 틀린 부분에 대해서 피드백을 주는 선생님이야.\n입력과 관련있는 정보를 참고해서 피드백을 생성해줘.\n참고한 정보의 페이지도 같이 알려줘.\n만약 틀린 부분이 없을 경우, 칭찬 한문장 작성해줘.\n\n#정보: \n{context}\n\n#입력:\n{question}\n\n#답:')
| ChatUpstage(client=<openai.resources.chat.completions.Completions object at 0x16d58bed0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x17d2fa510>, model_name='solar-pro', model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar')
| StrOutputParser()

In [26]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "정치는 개인과 개인, 개인과 집단, 집단과 집단 등 사회 구성원들 사이에서 발생하는 이해관계의 대립과 충돌을 해결하여 사회의 통합과 질서를 유지하는 행위를 뜻한다."

response = chain.invoke(question)
print(response)

잘 작성되었습니다! 정치의 의미를 다양한 관점에서 이해하고, 구체적 사례를 통해 정치의 기능을 설명하려는 학습 목표를 잘 반영하고 있습니다. 특히, 개인과 개인, 개인과 집단, 집단과 집단 등 사회 구성원들 사이에서 발생하는 이해관계의 대립과 충돌을 해결하여 사회의 통합과 질서를 유지하는 행위를 정치로 정의한 점이 정확합니다. (참고 문헌: politics_and_the_law_page_10-38.pdf)


In [27]:
question = "라스웰은 정치의 본질은 언제 어디서나 상반되는 가치와 감정을 동시에 포함한다는 데 있다 라고 했다"

response = chain.invoke(question)
print(response)

선생님: 라스웰이 정치의 본질에 대해 언급한 내용은 맞지만, 구체적인 문장은 뒤베르제가 말한 것입니다. 라스웰은 "정치란 가치의 권위적 배분이다"라고 정의했습니다. (출처: politics_and_the_law_page_10-38.pdf, page 2)


In [ ]:
# 기존에 생성된 벡터 DB를 불러와서 유사성 검색하기

pc = Pinecone()
index = pc.Index(index_name)
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

question = "라스웰은 정치의 본질은 언제 어디서나 상반되는 가치와 감정을 동시에 포함한다는 데 있다 라고 했다"
results = vectorstore.similarity_search(question, k=2)
results